# 🔄 Notebook: 01 ETL Silver Layer

This notebook handles the **Silver Layer (Data Transformation & Processing)** in the Medallion Architecture of the AI-powered claims pipeline. It focuses on **audio conversion**, **metadata extraction**, and **speech-to-text transcription** using the [OpenAI Whisper model](https://openai.com/index/whisper/).

---

## 🧱 Purpose

To convert raw audio files into a consistent format (MP3), calculate metadata (duration), and transcribe the content into structured text to support downstream AI analytics.



In [0]:
%pip install pydub mutagen openai-whisper numpy>=1.24
dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/800.5 kB ? eta -:--:--
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/800.5 kB 5.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━ 481.3/800.5 kB 7.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 798.7/800.5 kB 8.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/194.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/63.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.

In [0]:
%run "./resources/init" 

In [0]:
bronze_df = spark.table(f"{CATALOG}.{SCHEMA}.{BRONZE_TABLE}")

meta_table_name = f"{CATALOG}.{SCHEMA}.{META_TABLE}"
if spark._jsparkSession.catalog().tableExists(meta_table_name):
    processed_df = spark.table(meta_table_name).filter("processed = True")
    file_reference_df = bronze_df.join(processed_df, "file_name", "left_anti")
else:
    file_reference_df = bronze_df

if file_reference_df.isEmpty():
    dbutils.notebook.exit("✅ No new files to process. Exiting Silver Layer.")

In [0]:
from pydub import AudioSegment
import os

dbutils.fs.mkdirs(mp3_audio_path)

# Convert each file to mp3 and save to the new volume
for row in file_reference_df.collect():
    file_path = row['file_path']
    try:
        audio = AudioSegment.from_file(file_path)
        new_file_path = os.path.join(mp3_audio_path, os.path.basename(file_path).replace(os.path.splitext(file_path)[1], ".mp3"))
        audio.export(new_file_path, format="mp3")
    except Exception as e:
        print(f"⚠️ Error converting {file_path}: {e}")

In [0]:
from mutagen.mp3 import MP3
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

mp3_df = (
    spark.createDataFrame(dbutils.fs.ls(mp3_audio_path))
    .withColumn("file_path", F.expr("substring(path, 6, length(path))"))
    .withColumn("file_name", F.expr("substring(name, 1, length(name) - 4)"))
    .filter(F.col("file_name").isin([r["file_name"] for r in file_reference_df.collect()]))
)

def get_audio_duration(file_path):
    try:
        audio = MP3(file_path)
        return float(audio.info.length)
    except Exception as e:
        print(f"⚠️ Error getting duration for {file_path}: {e}")
        return None

duration_udf = F.udf(get_audio_duration, FloatType())
mp3_df = mp3_df.withColumn("audio_duration", F.round(duration_udf("file_path"), 0))

display(mp3_df)

path,name,size,modificationTime,file_path,file_name,audio_duration
dbfs:/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,724461,1744416267000,/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,91.0
dbfs:/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,865197,1744416258000,/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,108.0
dbfs:/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45.mp3,914349,1744416261000,/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,114.0
dbfs:/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,946605,1744416263000,/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33,118.0
dbfs:/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,956013,1744416265000,/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51,119.0


In [0]:
import whisper

model = whisper.load_model("small")
print("✅ Whisper model loaded.")

def transcribe_audio(file_path: str, model: whisper.Whisper) -> str:
    try:
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        print(f"⚠️ Transcription failed for {file_path}: {e}")
        return ""
      
from pyspark.sql.types import StringType
# from pyspark.sql.functions import udf

# transcribe_udf = udf(lambda path: transcribe_audio(path), StringType())

# transcribed_df = mp3_df.withColumn("transcription", transcribe_udf("file_path"))

# Collect the file paths to the driver
file_paths = mp3_df.select("file_path").rdd.flatMap(lambda x: x).collect()

# Transcribe the audio files outside of Spark
transcriptions = [transcribe_audio(file_path, model) for file_path in file_paths]

# Create a DataFrame with the transcriptions
transcriptions_df = spark.createDataFrame(zip(file_paths, transcriptions), ["file_path", "transcription"])

# Join the transcriptions back to the original DataFrame
transcribed_df = mp3_df.join(transcriptions_df, on="file_path", how="inner") \
                                 .select("file_path", "file_name", "transcription", "audio_duration")


Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 86.0MiB/s]
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f8485812-5c99-41f5-b9b6-f914659789ae/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

Model loaded successfully!


In [0]:
# if not first_run:
    

# display(transcribed_df) ####

/local_disk0/.ephemeral_nfs/envs/pythonEnv-f8485812-5c99-41f5-b9b6-f914659789ae/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f8485812-5c99-41f5-b9b6-f914659789ae/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f8485812-5c99-41f5-b9b6-f914659789ae/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f8485812-5c99-41f5-b9b6-f914659789ae/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  wa

file_path,file_name,transcription,audio_duration
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,"Hello, you've reached out Vital Guard, this is John speaking. How can I assist you today? Hi, this is Sophia Wilson. I was hoping to change my payment method. Of course, Sophia. For security, can I confirm your date of birth and policy number, please? Sure. My date of birth is 15th August 1995 and my policy number is VG924695. Thank you for confirming. What is your current payment method and how would you like to change it? I currently pay by direct debit, but I would like to switch to paying by credit card. Hmm, I can assist you with that. Please note that changing your payment method may affect your policy terms. Would you like me to explain the details? Yes, please do. Let me check that for you. Okay, I've updated your payment method to credit card. Your new payment schedule will be sent to you via email. Also, I'd like to take this opportunity to explain your coverage details. That sounds great, thank you. You're welcome. Just to summarize, I have changed your payment method to credit card and explained your coverage details. If you need further assistance, don't hesitate to reach out and have a wonderful day. Thanks, you too.",91.0
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,"Hello, this is Vital Guard, you are speaking to John, how can I help you today? Hi, this is Emma Johnson, I'm hoping to get some advice on my coverage options. Of course Emma, for security, can I confirm your date of birth and policy number please? Sure, my DOB is the 22nd of July 1990 and my policy number is VG 434271. Thank you for confirming. What kind of coverage details are you looking for? I want to know if my policy covers physiotherapy sessions. Okay, let me check that for you. Yes, your current plan includes up to 10 physiotherapy sessions per year, but they need to be prescribed by a GP. That's great to know. Also, if I need to make a complaint about a billing issue, how do I do that? I understand. I will provide you with a complaint reference number and you can submit your complaint through our online portal or over the phone. Here is your reference number, VG-CMP-4823. Thanks, that helps. Of course, you're very welcome. Is there anything else I can help you with today? No, that's all for now. Great. Just to summarize, Emma, I have confirmed your policy covers physiotherapy with a GP referral and I have provided a complaint reference number for your billing issue. If you do need further assistance, don't hesitate to reach out. Have a wonderful day. Thanks, you too.",108.0
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,"Hello, thank you for calling Vital Guard. My name is Michelle. How can I assist you today? Hey, this is Liam. Liam Brown. And I was hoping to get a duplicate policy document. Of course, Liam. For security, can I confirm your date of birth and policy number? Sure. It's 10th of May, 1978, and my policy number is VG82910. Thank you for confirming. I understand you're requesting a duplicate policy document. Can you please tell me what happened to the original document? I lost it during the move, and now I need it for my records. I apologize for the inconvenience, Liam. I'm going to go ahead and send you a duplicate policy document via email. You should receive it within the next 24 hours. Hmm, okay, that sounds okay. But I'm also a bit frustrated with the premium costs. Can you explain the breakdown? I completely understand your concern, Liam. I'd be happy to provide you with a premium breakdown. Your monthly premium is £3,100, which includes £350 for medical coverage and £320 for dental and £330 for adm

In [0]:
##### For simulated data purposes, not part of final solution

from pyspark.sql.functions import expr, substring

silver_table_path = f"{CATALOG}.{SCHEMA}.{SILVER_TABLE}"

if first_run:
    # Load existing transcriptions from the simulated table
    existing_transcriptions_df = spark.table(f"{CATALOG}.{SCHEMA}.simulated_transcriptions")
    
    # Extract file name from file path
    existing_transcriptions_df = existing_transcriptions_df.withColumn("file_name", expr("substring(file_path, -39, 35)"))
    
    # Combine existing (simulated) transcriptions with new transcriptions
    combined_transcriptions_df = existing_transcriptions_df.unionByName(transcribed_df)
    
    # Extract and transform relevant columns from file name
    combined_transcriptions_df = combined_transcriptions_df.withColumn("file_name", split(col("file_path"), "/").getItem(6)) \
        .withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4))))
    
    # Display the combined DataFrame
    display(combined_transcriptions_df)
    
    # Overwrite the transcriptions_silver table with the combined DataFrame
    combined_transcriptions_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)

else:
    transcribed_df = transcribed_df.withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4)))
        )

    display(transcribed_df.select("file_name", "call_id", "agent_id", "call_datetime", "audio_duration", "transcription"))

    if not spark._jsparkSession.catalog().tableExists(silver_table_path):
        transcribed_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)
    else:
        transcribed_df.write.mode("append").saveAsTable(silver_table_path)

    print(f"✅ Transcriptions written to Silver table: {silver_table_path}")

file_path,transcription,audio_duration,file_name,call_id,agent_id,call_datetime
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/262bf1a6_AGT003_2025-03-06 00:20:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Matthew Mitchell. I was hoping to get some advice on a claim I made. Of course, Matthew. For security, can I confirm your date of birth and policy number? Sure, my DOB is 16th January 1996, and my policy number is VG714141. Thank you for confirming. What kind of advice are you looking for regarding your claim? I want to know why it was flagged as potentially fraudulent. I understand. I will look into that for you... It appears that our system flagged your claim due to some discrepancies in the information provided. I will need to escalate this matter for further investigation. Okay, that makes sense. What happens next? I will escalate the suspected fraud and our investigations team will be in touch with you within the next 3-5 working days to discuss the matter further. In the meantime, please do not attempt to make any further claims. Alright, thank you for letting me know. You are very welcome. Is there anything else I can assist you with today? No, that is all for now. Great! Just to summarize, I have escalated the suspected fraud regarding your claim and our investigations team will be in touch soon. If you need further assistance, don’t hesitate to reach out. Have a wonderful day! Thanks! You too. Goodbye!",76.0,262bf1a6_AGT003_2025-03-06 00:20:38,262bf1a6,AGT003,2025-03-06T00:20:38Z
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/a9c22f7a_AGT003_2025-03-13 00:44:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Evelyn Harris. I was hoping to update my personal details. Of course, Evelyn. For security, can I confirm your date of birth and policy number? Sure, my DOB is 14th May 1958, and my policy number is VG630572. Thank you for confirming. What kind of personal details are you looking to update? I want to update my address. I can assist you with that. Can you please provide me with your new address? It's 123 Main Street, London. Thank you, Evelyn. I have updated your address to 123 Main Street, London. Is there anything else you would like to update? No, that's all for now. Great! Just to summarize, I have updated your address to 123 Main Street, London. If you need further assistance, don’t hesitate to reach out. Have a wonderful day!",134.0,a9c22f7a_AGT003_2025-03-13 00:44:38,a9c22f7a,AGT003,2025-03-13T00:44:38Z
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/dffc9b28_AGT002_2025-04-02 07:28:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Amelia Walker. I was hoping to find an in-network provider. Of course, Amelia. For security, can I confirm your date of birth and policy number? Sure, my DOB is 6th November 1981, and my policy number is VG412261. Thank you for confirming. What kind of provider are you looking for? I need to find a doctor in my area who is part of the VitalGuard network. Let me check that for you... Yes, I can provide you with a list of in-network providers in your area. Would you like me to email it to you or guide you through our online portal to find one? That would be great, thank you. Could you guide me through the online portal? Certainly. You can log in to your account on our website, click on 'Find a Provider,' and then enter your location. It will give you a list of all the in-network providers near you. Okay, got it. Is there anything else I can assist you with today? No, that's all for now. Great! Just to summarize, I have guided you through finding an in-network provider using our online portal. If you need further assistance, don't hesitate to reach out. Have a wonderful day! Thanks! You too. Goodbye!",99.0,df

## ✅ Output
- A clean, enriched Delta table: transcriptions_silver
- Includes transcription text, call metadata, and audio duration for each entry.